# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bilibino,68.0546,166.4372,-23.33,79,7,2.15,RU,1711847365
1,1,avarua,-21.2078,-159.7750,31.03,74,75,4.12,CK,1711847366
2,2,rio grande,-32.0350,-52.0986,21.81,85,0,2.26,BR,1711847290
3,3,nar'yan-mar,67.6713,53.0870,-15.50,93,75,3.13,RU,1711847366
4,4,thompson,55.7435,-97.8558,-8.29,93,100,3.17,CA,1711847351


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Define the plot using hvplot
city_map = city_data_df.hvplot.points(
    x='Lng', y='Lat', geo=True, size='Humidity', color='City', alpha = 0.5, tiles="OSM", hover_cols=["City", "Humidity"],  
    title='Humidity per City',  
    width=900, height=600) 


# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

# My ideal weather conditions: 
# Max temperature lower than 95 degrees but higher than 80
# Wind speed less than 4.5 m/s
# Zero cloudiness
# No preference on humidity

ideal_weather_df = city_data_df[(city_data_df['Max Temp'] < 35) & 
                           (city_data_df['Max Temp'] > 26) & 
                           (city_data_df['Wind Speed'] < 4.5) & 
                           (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
125,125,port augusta,-32.5000,137.7667,27.91,46,0,3.28,AU,1711847423
136,136,mahibadhoo,3.7833,72.9667,28.52,72,0,1.79,MV,1711847428
160,160,kone,-21.0595,164.8658,29.32,43,0,2.07,NC,1711847408
300,300,lom sak,16.7798,101.2422,29.93,57,0,2.80,TH,1711847504
303,303,mathba,22.2855,89.9629,26.51,81,0,2.81,BD,1711847506


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
125,port augusta,AU,-32.5000,137.7667,46,
136,mahibadhoo,MV,3.7833,72.9667,72,
160,kone,NC,-21.0595,164.8658,43,
300,lom sak,TH,16.7798,101.2422,57,
303,mathba,BD,22.2855,89.9629,81,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation",
    "limit": 1,
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
# get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
        
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port augusta - nearest hotel: Pampas Motel
mahibadhoo - nearest hotel: Bird Beak Beach
kone - nearest hotel: Pacifik Appartels
lom sak - nearest hotel: Sawang Hotel
mathba - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
pathein - nearest hotel: Taan Taan Ta
remire-montjoly - nearest hotel: Motel du Lac


,City,Country,Lat,Lng,Humidity,Hotel Name
125,port augusta,AU,-32.5000,137.7667,46,Pampas Motel
136,mahibadhoo,MV,3.7833,72.9667,72,Bird Beak Beach
160,kone,NC,-21.0595,164.8658,43,Pacifik Appartels
300,lom sak,TH,16.7798,101.2422,57,Sawang Hotel
303,mathba,BD,22.2855,89.9629,81,No hotel found
353,port hedland,AU,-20.3167,118.5667,43,The Esplanade Hotel
354,pathein,MM,16.7833,94.7333,56,Taan Taan Ta
365,remire-montjoly,GF,4.9167,-52.2667,94,Motel du Lac


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
ideal_weather_map = hotel_df.hvplot.points(
    x='Lng', y='Lat', geo=True, size='Humidity', color='City', alpha = 0.5, tiles="OSM", hover_cols=["City", "Country","Humidity", "Hotel Name"],  
    title='Ideal Weather Locations',  
    width=900, height=600) 


# Display the map
ideal_weather_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)